## Load Data and import necesary packages

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs('checkpoints/', exist_ok = True)
checkpoint_path_1 = 'checkpoints/weights-opt-1.{epoch:02d}.hdf5'
checkpoint_path_2 = 'checkpoints/weights-opt-2.{epoch:02d}.hdf5'
checkpoint_path_3 = 'checkpoints/weights-opt-3.{epoch:02d}.hdf5'

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv('./Resources/charity_data.csv')

## Design functions to automatically clean data, create the model, and run it to simplify sweeping the parameter space for optimization

In [2]:
def run_model (data, drop = ['EIN', 'NAME'], replace = [
            { 'col': 'APPLICATION_TYPE', 'threshold': 500 },
            { 'col': 'CLASSIFICATION', 'threshold': 1000 }
        ], input_node = { 'units': 80, 'activation': 'relu' },
        hidden_nodes = [{ 'units': 30, 'activation': 'relu' }],
        output_node = { 'units': 1, 'activation': 'sigmoid' },
        optimizer = 'adam',
        epochs = 100, save = False, save_name = '', path = ''):
    
    # clone data
    app = data.copy()
    
    # clean data
    app = clean_data(app, drop, replace)
    
    #get train/test data and make model
    X_train, X_test, y_train, y_test = get_target_and_features(app)
    model = create_model(X_train, input_node, hidden_nodes, output_node, optimizer)
    
    #run and evaluate model
    evaluate(model, X_train, X_test, y_train, y_test, epochs, save, save_name, path)
    


In [3]:
def clean_data(app, drop, replace):
    # drop columns
    app = app.drop(drop, 1)
    
    # compress large sets of data
    for rep in replace:
        app = compress(app, rep['col'], rep['threshold'])
        
    #replace cats as dummys
    return encode(app)

In [4]:
def compress (data, col, threshold):
    # clone data
    app = data.copy()
    
    # get vals to replace
    counts = app[col].value_counts()
    replace = list(counts[counts < threshold].index)
    
    #replace the values and return
    for rep in replace:
        app[col] = app[col].replace(rep, 'Other')
    return app
    

In [5]:
def encode(data):
    # clone data
    app = data.copy()
    
    # get cats     
    cat = app.dtypes[app.dtypes == 'object'].index.tolist()
    
    # make encoder and encode
    enc = OneHotEncoder(sparse = False)
    encode = pd.DataFrame(enc.fit_transform(app[cat]))
    encode.columns = enc.get_feature_names(cat)    
    
    # return new df
    return app.merge(encode, left_index = True, right_index = True).drop(cat, 1)


In [6]:
def get_target_and_features(app):
    # Split our preprocessed data into our features and target arrays
    y = app['IS_SUCCESSFUL'].values
    X = app.drop(['IS_SUCCESSFUL'], 1).values

    # Split the preprocessed data into a training and testing dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 78)
    
    # Create a StandardScaler instances
    scaler = StandardScaler()

    # Fit the StandardScaler
    X_scaler = scaler.fit(X_train)

    # Scale the data
    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)
    
    return X_train_scaled, X_test_scaled, y_train, y_test

In [7]:
def create_model(X_train, input_node, hidden_nodes, output_node, optimizer):

    # get the input node number
    num_features = len(X_train[0])
    
    # create model
    nn = tf.keras.models.Sequential()
    
    #add layers
    #input
    nn.add(tf.keras.layers.Dense(units = input_node['units'], input_dim = num_features,
                                 activation = input_node['activation']))

    #hidden
    for node in hidden_nodes:
        nn.add(tf.keras.layers.Dense(units = node['units'], activation = node['activation']))
        
    #output
    nn.add(tf.keras.layers.Dense(units = output_node['units'], activation = output_node['activation']))

    #compile and return
    nn.compile(loss = 'binary_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
    return nn


In [8]:
def create_callback (path):
    return ModelCheckpoint(
        filepath = path,
        verbose = 1,
        save_weights_only = True,
        save_freq = 5 * 804)

In [9]:
def evaluate(model, X_train, X_test, y_train, y_test, epochs, save, save_name, path):
    # Final model run
    if save:
        fit_model = model.fit(X_train, y_train, epochs = epochs, callbacks = [create_callback(path)])
        model_loss, model_accuracy = model.evaluate(X_test, y_test, verbose = 2)
        print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
        model.save(f"AlphabetSoupCharity_Optimization_{save_name}.h5")
    # Quiet optimization run
    else:
        fit_model = model.fit(X_train, y_train, epochs = epochs, verbose = 0)
        model_loss, model_accuracy = model.evaluate(X_test, y_test, verbose = 0)
        print(f"Accuracy: {model_accuracy:02f}")
    

In [10]:
def check_best_weights(data, drop = ['EIN', 'NAME'], replace = [
            { 'col': 'APPLICATION_TYPE', 'threshold': 500 },
            { 'col': 'CLASSIFICATION', 'threshold': 1000 }
        ], input_node = { 'units': 80, 'activation': 'relu' },
        hidden_nodes = [{ 'units': 30, 'activation': 'relu' }],
        output_node = { 'units': 1, 'activation': 'sigmoid' },
        optimizer = 'adam',
        epochs = 100, save = False, save_name = '', path = ''):
    
    # clone data
    app = data.copy()
    
    # clean data
    app = clean_data(app, drop, replace)
    
    #get train/test data and make model
    X_train, X_test, y_train, y_test = get_target_and_features(app)
    model = create_model(X_train, input_node, hidden_nodes, output_node, optimizer)
    model.load_weights(path)
    
    # evaluate model
    model_loss, model_accuracy = model.evaluate(X_test, y_test, verbose = 2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
    if save:
        model.save(f"AlphabetSoupCharity_Optimization_{save_name}.h5")
    return model_accuracy

## Sweeps

### Rerun Original model to test it functions work as expected
#### (The original model is defined by the parameter defaults)
Accuracy: 0.724665

In [11]:
# run_model(application_df)

### 1) remove each non-automatically optimized, dropped, or target col in turn

#### Results:
##### AFFILIATION

Trial 1 - Accuracy: 0.655977

Trial 2 - Accuracy: 0.658309

Affiliation appears to be an important variable.

##### USE_CASE

Trial 1 - Accuracy: 0.726297

Trial 2 - Accuracy: 0.726531

Use case appears to be not particularly important

#####  ORGANIZATION
Trial 1 - Accuracy: 0.723149

Trial 2 - Accuracy: 0.721982

Organization appears to be somewhat important

##### STATUS
Trial 1 - Accuracy: 0.726181

Trial 2 - Accuracy: 0.725131

Status appears to be not particularly important

##### INCOME_AMT

Trial 1 - Accuracy: 0.727580

Trial 2 - Accuracy: 0.727114

Income appears to be not important

#####  SPECIAL_CONSIDERATIONS

Trial 1 - Accuracy: 0.726997

Trial 2 - Accuracy: 0.725714

Special considerations appears to be not particularly important

#####  ASK_AMT

Trial 1 - Accuracy: 0.726531 

Trial 2 - Accuracy: 0.725364

Ask amt appears to be not particularly important

In [12]:
# cols = application_df.columns
# test_remove = [col for col in cols if col not in
#                ['EIN', 'NAME', 'APPLICATION_TYPE', 'CLASSIFICATION', 'IS_SUCCESSFUL']]

# for remove in test_remove:
#     print(remove)
#     run_model(application_df, drop = ['EIN', 'NAME', remove])

#### test removal of all columns deems not particularly important
Accuracy: 0.723032

In [13]:
# run_model(application_df, drop = [ 'EIN', 'NAME',
#   'USE_CASE', 'STATUS', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS', 'ASK_AMT'
# ])

#### test removal of some columns deemed not particularly important
Accuracy: 0.726997

In [14]:
# run_model(application_df, drop = [ 'EIN', 'NAME',
#   'USE_CASE', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'
# ])

#### Conclusion:
remove 'USE_CASE', 'INCOME_AMT', and 'SPECIAL_CONSIDERATIONS'

check the data for AFFILIATION further

In [15]:
application_df['AFFILIATION'].value_counts()

Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: AFFILIATION, dtype: int64

### 2) Test model with affiliation as only 3 categories
trial 1 - Accuracy: 0.726880

trial 2 - Accuracy: 0.728513

conclusion: this doesn't hurt the model and the upper range of accuracy has increased.  Keep the three categories

In [16]:
# run_model(application_df, drop = [ 'EIN', 'NAME', 'USE_CASE', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'],
#          replace = [
#             { 'col': 'APPLICATION_TYPE', 'threshold': 500 },
#             { 'col': 'CLASSIFICATION', 'threshold': 1000 },
#          { 'col': 'AFFILIATION', 'threshold': 100 }])

### 3) Test lower thresholds for app type and class

#### Results 1
##### Application Type
100 trial 1 - Accuracy: 0.729329

100 trial 2 - Accuracy: 0.728630

250  trial 1 - Accuracy: 0.728863

250  trial 2 - Accuracy: 0.727813

500 trial 1 - Accuracy: 0.728863

500 trial 2 - Accuracy: 0.727114

1000 trial 1 - Accuracy: 0.722915

1000 trial 2 - Accuracy: 0.724665

1500 trial 1 - Accuracy: 0.715569

1500 trial 2 - Accuracy: 0.718134

2000 trial 1 - Accuracy: 0.701225

2000 trial 2 - Accuracy: 0.703324

##### Classification
100 trial 1 - Accuracy: 0.727347

100 trial 2 - Accuracy: 0.727697

250 trial 1 - Accuracy: 0.728280

250  trial 2 - Accuracy: 0.727464

500 trial 1 - Accuracy: 0.727230

500 trial 2 - Accuracy: 0.727580

1000 trial 1 - Accuracy: 0.728513

1000 trial 2 - Accuracy: 0.727813

1500 trial 1 - Accuracy: 0.728513

1500 trial 2 - Accuracy: 0.728280

2000 trial 1 - Accuracy: 0.726647

2000 trial 2 - Accuracy: 0.725248

#### Conclusions
Application type definitely performs better when the data is more granular.  Test a few more threholds around 100.

Classification seems to be about the same regardless of the threshold.  It's marginally the best between 100-1500, which is where it currently is.  Maybe test in that region a bit more, but optimizing it will lead to modest improvements.

In [17]:
# newThreholds = [100, 250, 500, 1000, 1500, 2000]

# for thresh in newThreholds:
#     print(f"APPLICATION_TYPE: {thresh}")
#     run_model(application_df, drop = [ 'EIN', 'NAME', 'USE_CASE', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'],
#              replace = [
#                 { 'col': 'APPLICATION_TYPE', 'threshold': thresh },
#                 { 'col': 'CLASSIFICATION', 'threshold': 1000 },
#              { 'col': 'AFFILIATION', 'threshold': 100 }])
    
# for thresh in newThreholds:
#     print(f"CLASSIFICATION: {thresh}")
#     run_model(application_df, drop = [ 'EIN', 'NAME', 'USE_CASE', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'],
#              replace = [
#                 { 'col': 'APPLICATION_TYPE', 'threshold': 500 },
#                 { 'col': 'CLASSIFICATION', 'threshold': thresh },
#              { 'col': 'AFFILIATION', 'threshold': 100 }])

#### Results 2
##### Application Type
25 trial 1 - Accuracy: 0.728163

25 trial 2 - Accuracy: 0.729213

50 trial 1 - Accuracy: 0.726764

50 trial 2 - Accuracy: 0.727347

100 trial 1 - Accuracy: 0.727697

100 trial 2 - Accuracy: 0.728630

200 trial 1 - Accuracy: 0.729563

200 trial 2 - Accuracy: 0.727930

##### Classification
1100 trial 1 - Accuracy: 0.726064

1100 trial 2 - Accuracy: 0.726764

1200 trial 1 - Accuracy: 0.726181

1200 trial 2 - Accuracy: 0.730262

1300 trial 1 - Accuracy: 0.728746

1300 trial 2 - Accuracy: 0.728047

1400 trial 1 - Accuracy: 0.729796

1400 trial 2 - Accuracy: 0.728163

### Conclusions:
It's a tough choice between 25 and 200 for app type, but app type got the highest value for that optimization, and it redcues the number of columns more.  Going to try 200 from now on.

I'm glad I tried to optimize classification further.  I am very happy with the results at 1200 and 1400.  I going to go with 1400 because it was more consistent and leads to fewer columns.

In [18]:
# app_threholds = [25, 50, 100, 200]

# for thresh in app_threholds:
#     print(f"APPLICATION_TYPE: {thresh}")
#     run_model(application_df, drop = [ 'EIN', 'NAME', 'USE_CASE', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'],
#              replace = [
#                 { 'col': 'APPLICATION_TYPE', 'threshold': thresh },
#                 { 'col': 'CLASSIFICATION', 'threshold': 1000 },
#              { 'col': 'AFFILIATION', 'threshold': 100 }])
 
# class_threholds = c

# for thresh in class_threholds:
#     print(f"CLASSIFICATION: {thresh}")
#     run_model(application_df, drop = [ 'EIN', 'NAME', 'USE_CASE', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'],
#              replace = [
#                 { 'col': 'APPLICATION_TYPE', 'threshold': 500 },
#                 { 'col': 'CLASSIFICATION', 'threshold': thresh },
#              { 'col': 'AFFILIATION', 'threshold': 100 }])

### 4) Run both threshold optimizations and increase training time to ensure model converges
#### results
##### with 100 epochs
trial 1 - Accuracy: 0.726764

trial 2 - Accuracy: 0.727813

trial 3 - Accuracy: 0.728047

##### with 300 epochs
trial 1 - Accuracy: 0.727230

trial 2 - Accuracy: 0.726764

#### conclusion:
There must be a relationship between these variables, such optimizing one alone is not sufficient; they affect one another.  I don't want to run the two together, so I'm going to just use the classification optimization.

In [19]:
#     run_model(application_df, drop = [ 'EIN', 'NAME', 'USE_CASE', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'],
#              replace = [
#                 { 'col': 'APPLICATION_TYPE', 'threshold': 200 },
#                 { 'col': 'CLASSIFICATION', 'threshold': 1400 },
#              { 'col': 'AFFILIATION', 'threshold': 100 }],
#              epochs = 300)

### 5) Run just classification and increase training time to ensure model converges
Trial 1 - Accuracy: 0.729446

Trial 2 - Accuracy: 0.727813

In [20]:
# run_model(application_df, drop = [ 'EIN', 'NAME', 'USE_CASE', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'],
#          replace = [
#             { 'col': 'APPLICATION_TYPE', 'threshold': 500 },
#             { 'col': 'CLASSIFICATION', 'threshold': 1400 },
#          { 'col': 'AFFILIATION', 'threshold': 100 }],
#          epochs = 300)

### 6) Optimize activator functions
#### Input: relu, Hidden: sigmoid, Output: tanh

trial 1 - Accuracy: 0.729679

trial 2 - Accuracy: 0.727230

#### Input: relu, Hidden: sigmoid, Output: leaky_relu

trial 1 - Accuracy: 0.725714

trial 2 - Accuracy: 0.728630

#### Input: relu, Hidden: tanh, Output: sigmoid

trial 1 - Accuracy: 0.728863

trial 2 - Accuracy: 0.727930

#### Input: relu, Hidden: tanh, Output: leaky_relu

trial 1 - Accuracy: 0.726181

trial 2 - Accuracy: 0.727580

#### Input: relu, Hidden: leaky_relu, Output: sigmoid

trial 1 - Accuracy: 0.728163

trial 2 - Accuracy: 0.728863

#### Input: relu, Hidden: leaky_relu, Output: tanh

trial 1 - Accuracy: 0.728163

trial 2 - Accuracy: 0.724548

#### Input: sigmoid, Hidden: relu, Output: tanh

trial 1 - Accuracy: 0.725947

trial 2 - Accuracy: 0.724082

#### Input: sigmoid, Hidden: relu, Output: leaky_relu

trial 1 - Accuracy: 0.725598

trial 2 - Accuracy: 0.726647

#### Input: sigmoid, Hidden: tanh, Output: relu

trial 1 - Accuracy: 0.466822

trial 2 - Accuracy: 0.724198

#### Input: sigmoid, Hidden: tanh, Output: leaky_relu

trial 1 - Accuracy: 0.726064

trial 2 - Accuracy: 0.725131

#### Input: sigmoid, Hidden: leaky_relu, Output: relu

trial 1 - Accuracy: 0.723965

trial 2 - Accuracy: 0.533178

#### Input: sigmoid, Hidden: leaky_relu, Output: tanh

trial 1 - Accuracy: 0.727347

trial 2 - Accuracy: 0.723965

#### Input: tanh, Hidden: relu, Output: sigmoid

trial 1 - Accuracy: 0.727580

trial 2 - Accuracy: 0.728047

#### Input: tanh, Hidden: relu, Output: leaky_relu

trial 1 - Accuracy: 0.725598

trial 2 - Accuracy: 0.726064

#### Input: tanh, Hidden: sigmoid, Output: relu

trial 1 - Accuracy: 0.728163

trial 2 - Accuracy: 0.726414

#### Input: tanh, Hidden: sigmoid, Output: leaky_relu

trial 1 - Accuracy: 0.466822

trial 2 - Accuracy: 0.727580

#### Input: tanh, Hidden: leaky_relu, Output: relu

trial 1 - Accuracy: 0.726181

trial 2 - Accuracy: 0.727697

#### Input: tanh, Hidden: leaky_relu, Output: sigmoid

trial 1 - Accuracy: 0.727813

trial 2 - Accuracy: 0.727230

#### Input: leaky_relu, Hidden: relu, Output: sigmoid

trial 1 - Accuracy: 0.727697

trial 2 - Accuracy: 0.726880

#### Input: leaky_relu, Hidden: relu, Output: tanh

trial 1 - Accuracy: 0.726764

trial 2 - Accuracy: 0.725131

#### Input: leaky_relu, Hidden: sigmoid, Output: relu

trial 1 - Accuracy: 0.466822

trial 2 - Accuracy: 0.725714

#### Input: leaky_relu, Hidden: sigmoid, Output: tanh

trial 1 - Accuracy: 0.466822

trial 2 - Accuracy: 0.727347

#### Input: leaky_relu, Hidden: tanh, Output: relu

trial 1 - Accuracy: 0.728047

trial 2 - Accuracy: 0.727230

#### Input: leaky_relu, Hidden: tanh, Output: sigmoid

trial 1 - Accuracy: 0.726531

trial 2 - Accuracy: 0.727930

### conclusions:
Good models were those that have at least .727 accuracy on both trials.  Sigmoid seems to be the best choice for the output node based on all good models.  All good models had a tahn and a relu or leaky_relu layer, where relu models performed better.  Good models also tended to perform better when the input node had relu or leaky relu versus tahn.  I need to test 5 more models: all nodes of the same type (4) and tanh tanh sigmoid for comparison against relu relu sigmoid results above.


In [21]:
from itertools import permutations

In [22]:
# activator_functions = ['relu', 'sigmoid', 'tanh', 'leaky_relu']
# # this is 24 models
# activators = list(permutations(activator_functions, 3))

# for act in activators:
#     print(f"Input: {act[0]}, Hidden: {act[1]}, Output: {act[2]}")
#     run_model(application_df, drop = [ 'EIN', 'NAME', 'USE_CASE', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'],
#              replace = [
#                 { 'col': 'APPLICATION_TYPE', 'threshold': 500 },
#                 { 'col': 'CLASSIFICATION', 'threshold': 1400 },
#                  { 'col': 'AFFILIATION', 'threshold': 100 }
#              ],
#               input_node = { 'units': 80, 'activation': act[0] },
#             hidden_nodes = [{ 'units': 30, 'activation':  act[1]}],
#             output_node = { 'units': 1, 'activation':  act[2]}
#              )

### 7) Optimize activator functions part 2
#### Relu relu sigmoid results from 5
Trial 1 - Accuracy: 0.729446

Trial 2 - Accuracy: 0.727813

#### relu tanh sigmoid results from 6

trial 1 - Accuracy: 0.728863

trial 2 - Accuracy: 0.727930

#### Input: relu, Hidden: relu, Output: relu

Trial 1 - Accuracy: 0.724781

Trial 2 - Accuracy: 0.727230

#### Input: sigmoid, Hidden: sigmoid, Output: sigmoid

Trial 1 - Accuracy: 0.726414

Trial 2 - Accuracy: 0.727580

#### Input: tanh, Hidden: tanh, Output: tanh

Trial 1 - Accuracy: 0.722682

Trial 2 - Accuracy: 0.726531

#### Input: leaky_relu, Hidden: leaky_relu, Output: leaky_relu

Trial 1 - Accuracy: 0.723848

Trial 2 - Accuracy: 0.721982

#### Input: tanh, Hidden: tanh, Output: sigmoid

Trial 1 - Accuracy: 0.726997

Trial 2 - Accuracy: 0.726997

### Conclusions:
Relu relu sigmoid looks like the best choice.  I will keep adding some tanh to the hidden nodes in mind while expanding the neural network.



In [23]:
# activator_functions = ['relu', 'sigmoid', 'tanh', 'leaky_relu']
# activators = [
#     ['relu', 'relu', 'relu'],
#     ['sigmoid', 'sigmoid', 'sigmoid'],
#     ['tanh', 'tanh', 'tanh'],
#     ['leaky_relu', 'leaky_relu', 'leaky_relu'],
#     ['tanh', 'tanh', 'sigmoid']
# ]

# for act in activators:
#     print(f"Input: {act[0]}, Hidden: {act[1]}, Output: {act[2]}")
#     run_model(application_df, drop = [ 'EIN', 'NAME', 'USE_CASE', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'],
#              replace = [
#                 { 'col': 'APPLICATION_TYPE', 'threshold': 500 },
#                 { 'col': 'CLASSIFICATION', 'threshold': 1400 },
#                  { 'col': 'AFFILIATION', 'threshold': 100 }
#              ],
#               input_node = { 'units': 80, 'activation': act[0] },
#             hidden_nodes = [{ 'units': 30, 'activation':  act[1]}],
#             output_node = { 'units': 1, 'activation':  act[2]}
#              )

### 8) Changing node numbers
#### results
##### Nodes: 2, 1

trial 1 - Accuracy: 0.533178

trial 2 - Accuracy: 0.533178

##### Nodes: 4, 2

trial 1 - Accuracy: 0.723965

trial 2 - Accuracy: 0.723965

##### Nodes: 8, 4

trial 1 - Accuracy: 0.725714

trial 2 - Accuracy: 0.726181

##### Nodes: 16, 8

trial 1 - Accuracy: 0.728513

trial 2 - Accuracy: 0.727697

##### Nodes: 32, 16

trial 1 - Accuracy: 0.727464

trial 2 - Accuracy: 0.727580

##### Nodes: 64, 32

trial 1 - Accuracy: 0.726531

trial 2 - Accuracy: 0.727813

##### Nodes: 128, 64

trial 1 - Accuracy: 0.728047

trial 2 - Accuracy: 0.728396

##### Nodes: 256, 128

trial 1 - Accuracy: 0.726880

trial 2 - Accuracy: 0.728280

### Conclusions
Increasing the number of neurons really didn't help much.  Decreasing them did eventually hurt the model.  I might just go with 128/64 because it reached .728 more consistently.

In [24]:
# nNodes_list = [
#     [2, 1],
#     [4, 2],
#     [8, 4],
#     [16, 8],
#     [32, 16],
#     [64, 32],
#     [128, 64],
#     [256, 128]
# ]

# for nNodes in nNodes_list:
#     print(f"Nodes: {nNodes[0]}, {nNodes[1]}")
#     run_model(application_df, drop = [ 'EIN', 'NAME', 'USE_CASE', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'],
#              replace = [
#                 { 'col': 'APPLICATION_TYPE', 'threshold': 500 },
#                 { 'col': 'CLASSIFICATION', 'threshold': 1400 },
#                  { 'col': 'AFFILIATION', 'threshold': 100 }
#              ],
#               input_node = { 'units': nNodes[0], 'activation': 'relu' },
#             hidden_nodes = [{ 'units': nNodes[1], 'activation':  'relu'}],
#             output_node = { 'units': 1, 'activation':  'sigmoid'}
#              )

### 9) Add a new layer
#### results
##### Relu relu relu sigmoid
trial 1 - Accuracy: 0.727697

trial 2 - Accuracy: 0.725598

##### Relu relu tanh sigmoid
trial 1 - Accuracy: 0.729913

trial 2 - Accuracy: 0.726880

#### Conclusions
It's really not apparent whether adding another layer actually helped, but relu-relu-tahn-sigmoid perormed well, so I'll continue with it.



In [25]:
# print('Relu relu relu sigmoid')
# run_model(application_df, drop = [ 'EIN', 'NAME', 'USE_CASE', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'],
#          replace = [
#             { 'col': 'APPLICATION_TYPE', 'threshold': 500 },
#             { 'col': 'CLASSIFICATION', 'threshold': 1400 },
#              { 'col': 'AFFILIATION', 'threshold': 100 }
#          ],
#           input_node = { 'units': 128, 'activation': 'relu' },
#         hidden_nodes = [
#             { 'units': 64, 'activation':  'relu'},
#             { 'units': 32, 'activation':  'relu'}
#                        ],
#         output_node = { 'units': 1, 'activation':  'sigmoid'}
#          )
    
# print('Relu relu tanh sigmoid')
# run_model(application_df, drop = [ 'EIN', 'NAME', 'USE_CASE', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'],
#          replace = [
#             { 'col': 'APPLICATION_TYPE', 'threshold': 500 },
#             { 'col': 'CLASSIFICATION', 'threshold': 1400 },
#              { 'col': 'AFFILIATION', 'threshold': 100 }
#          ],
#           input_node = { 'units': 128, 'activation': 'relu' },
#         hidden_nodes = [
#             { 'units': 64, 'activation':  'relu'},
#             { 'units': 32, 'activation':  'tanh'}
#                        ],
#         output_node = { 'units': 1, 'activation':  'sigmoid'}
#          )

### 10) Add a new layer part 2
Trial 1 - Accuracy: 0.728513

Trial 2 - Accuracy: 0.727813

Trial 3 - Accuracy: 0.727580

Trial 4 - Accuracy: 0.728396

#### Conclusions
It is extremely unclear if this is helping at all. I don't think it makes sense to add the fourth layer.

In [26]:
# for i in [1, 2]:
#     print(f"Trial {i} - ")
#     run_model(application_df, drop = [ 'EIN', 'NAME', 'USE_CASE', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'],
#              replace = [
#                 { 'col': 'APPLICATION_TYPE', 'threshold': 500 },
#                 { 'col': 'CLASSIFICATION', 'threshold': 1400 },
#                  { 'col': 'AFFILIATION', 'threshold': 100 }
#              ],
#               input_node = { 'units': 128, 'activation': 'relu' },
#             hidden_nodes = [
#                 { 'units': 64, 'activation':  'relu'},
#                 { 'units': 32, 'activation':  'tanh'},
#                 { 'units': 16, 'activation':  'tanh'}
#                            ],
#             output_node = { 'units': 1, 'activation':  'sigmoid'}
#              )

### 11) Changing the optimizing function
#### results
##### Use: adadelta
Trial 1 - Accuracy: 0.721283

Trial 2 - Accuracy: 0.721982

##### Use: adagrad
Trial 1 - Accuracy: 0.723032

Trial 2 - Accuracy: 0.722682

##### Use: adam
Trial 1 - Accuracy: 0.728863

Trial 2 - Accuracy: 0.728047

##### Use: adamax
Trial 1 - Accuracy: 0.727697

Trial 2 - Accuracy: 0.727114

##### Use: ftrl
Trial 1 - Accuracy: 0.533178

Trial 2 - Accuracy: 0.533178

##### Use: nadam
Trial 1 - Accuracy: 0.728513

Trial 2 - Accuracy: 0.727347

##### Use: rmsprop
Trial 1 - Accuracy: 0.726764

Trial 2 - Accuracy: 0.726181

##### Use: sgd
Trial 1 - Accuracy: 0.723848

Trial 2 - Accuracy: 0.723615

#### Conclusions
Adam, the model defualt, seems to be the best choice for the optimizing function

In [27]:
# optimizing_functions = ['adadelta', 'adagrad', 'adam', 'adamax', 'ftrl', 'nadam', 'rmsprop', 'sgd']

# for func in optimizing_functions:
#     print(f"Use: {func}")
#     run_model(application_df, drop = [ 'EIN', 'NAME', 'USE_CASE', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'],
#              replace = [
#                 { 'col': 'APPLICATION_TYPE', 'threshold': 500 },
#                 { 'col': 'CLASSIFICATION', 'threshold': 1400 },
#                  { 'col': 'AFFILIATION', 'threshold': 100 }
#              ],
#               input_node = { 'units': 128, 'activation': 'relu' },
#             hidden_nodes = [
#                 { 'units': 64, 'activation':  'relu'},
#                 { 'units': 32, 'activation':  'tanh'}
#                            ],
#             output_node = { 'units': 1, 'activation':  'sigmoid'},
#             optimizer = func
#              )

## Giving Up
At this point, given how difficult it has been to even breach 73% accurate, I'm not sure if 75% accurate is even possible, or if that goal was a trap set to make perfectionists waste time.

### Best Attempt 1
Two hidden layers, use relu and tanh mixed for hidden layers, train 500 epochs, drop USE_CASE', 'INCOME_AMT', and 'SPECIAL_CONSIDERATIONS, use 128, 64 and 32 neurons.

In [28]:
run_model(application_df, drop = [ 'EIN', 'NAME', 'USE_CASE', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS', 'STATUS'],
         replace = [
            { 'col': 'APPLICATION_TYPE', 'threshold': 500 },
            { 'col': 'CLASSIFICATION', 'threshold': 1400 },
             { 'col': 'AFFILIATION', 'threshold': 100 }
         ],
          input_node = { 'units': 128, 'activation': 'relu' },
        hidden_nodes = [
            { 'units': 64, 'activation':  'relu'},
            { 'units': 32, 'activation':  'tanh'}
                       ],
        output_node = { 'units': 1, 'activation':  'sigmoid'},
        epochs = 500, save = True, path = checkpoint_path_1,
        save_name = '1')

Epoch 1/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5720 - accuracy: 0.7205
Epoch 2/500
804/804 [==============================] - 1s 2ms/step - loss: 0.5654 - accuracy: 0.7260
Epoch 3/500
804/804 [==============================] - 1s 2ms/step - loss: 0.5641 - accuracy: 0.7261
Epoch 4/500
804/804 [==============================] - 1s 2ms/step - loss: 0.5636 - accuracy: 0.7272
Epoch 5/500
760/804 [===========================>..] - ETA: 0s - loss: 0.5625 - accuracy: 0.7287
Epoch 00005: saving model to checkpoints\weights-opt-1.05.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5630 - accuracy: 0.7282
Epoch 6/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5620 - accuracy: 0.7277
Epoch 7/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5609 - accuracy: 0.7282
Epoch 8/500
804/804 [==============================] - 1s 2ms/step - loss: 0.5606 - accuracy: 0.7283
Epoch 9/500
804/804 [=======================

804/804 [==============================] - 1s 1ms/step - loss: 0.5541 - accuracy: 0.7327
Epoch 64/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5540 - accuracy: 0.7330
Epoch 65/500
801/804 [============================>.] - ETA: 0s - loss: 0.5541 - accuracy: 0.7331
Epoch 00065: saving model to checkpoints\weights-opt-1.65.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5542 - accuracy: 0.7331
Epoch 66/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5548 - accuracy: 0.7316
Epoch 67/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5545 - accuracy: 0.7315
Epoch 68/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5550 - accuracy: 0.7322
Epoch 69/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5542 - accuracy: 0.7318
Epoch 70/500
765/804 [===========================>..] - ETA: 0s - loss: 0.5539 - accuracy: 0.7322
Epoch 00070: saving model to checkpoints\weights-opt-

781/804 [============================>.] - ETA: 0s - loss: 0.5533 - accuracy: 0.7320
Epoch 00125: saving model to checkpoints\weights-opt-1.125.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5534 - accuracy: 0.7319
Epoch 126/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5538 - accuracy: 0.7317
Epoch 127/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5532 - accuracy: 0.7333
Epoch 128/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5534 - accuracy: 0.7331
Epoch 129/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5529 - accuracy: 0.7322
Epoch 130/500
794/804 [============================>.] - ETA: 0s - loss: 0.5524 - accuracy: 0.7334
Epoch 00130: saving model to checkpoints\weights-opt-1.130.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5531 - accuracy: 0.7329
Epoch 131/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5536 - accuracy: 0.

Epoch 186/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5524 - accuracy: 0.7334
Epoch 187/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5525 - accuracy: 0.7334
Epoch 188/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5524 - accuracy: 0.7329
Epoch 189/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5526 - accuracy: 0.7329
Epoch 190/500
787/804 [============================>.] - ETA: 0s - loss: 0.5524 - accuracy: 0.7334
Epoch 00190: saving model to checkpoints\weights-opt-1.190.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5526 - accuracy: 0.7334
Epoch 191/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5527 - accuracy: 0.7331
Epoch 192/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5526 - accuracy: 0.7335
Epoch 193/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5528 - accuracy: 0.7338
Epoch 194/500
804/804 [====

804/804 [==============================] - 1s 1ms/step - loss: 0.5532 - accuracy: 0.7320
Epoch 249/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5523 - accuracy: 0.7338
Epoch 250/500
791/804 [============================>.] - ETA: 0s - loss: 0.5524 - accuracy: 0.7332
Epoch 00250: saving model to checkpoints\weights-opt-1.250.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5524 - accuracy: 0.7332
Epoch 251/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5538 - accuracy: 0.7332
Epoch 252/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5525 - accuracy: 0.7336
Epoch 253/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5523 - accuracy: 0.7332
Epoch 254/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5535 - accuracy: 0.7311
Epoch 255/500
784/804 [============================>.] - ETA: 0s - loss: 0.5534 - accuracy: 0.7323
Epoch 00255: saving model to checkpoints\weig

791/804 [============================>.] - ETA: 0s - loss: 0.5541 - accuracy: 0.7329
Epoch 00310: saving model to checkpoints\weights-opt-1.310.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5536 - accuracy: 0.7336
Epoch 311/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5529 - accuracy: 0.7328
Epoch 312/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5535 - accuracy: 0.7342
Epoch 313/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5528 - accuracy: 0.7338
Epoch 314/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5555 - accuracy: 0.7333
Epoch 315/500
771/804 [===========================>..] - ETA: 0s - loss: 0.5544 - accuracy: 0.7332
Epoch 00315: saving model to checkpoints\weights-opt-1.315.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5549 - accuracy: 0.7325
Epoch 316/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5533 - accuracy: 0.

804/804 [==============================] - 1s 1ms/step - loss: 0.5530 - accuracy: 0.7327
Epoch 371/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5522 - accuracy: 0.7331
Epoch 372/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5539 - accuracy: 0.7332
Epoch 373/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5535 - accuracy: 0.7321
Epoch 374/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5553 - accuracy: 0.7320
Epoch 375/500
761/804 [===========================>..] - ETA: 0s - loss: 0.5538 - accuracy: 0.7315
Epoch 00375: saving model to checkpoints\weights-opt-1.375.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5536 - accuracy: 0.7320
Epoch 376/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5527 - accuracy: 0.7329
Epoch 377/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5536 - accuracy: 0.7329
Epoch 378/500
804/804 [==================

804/804 [==============================] - 1s 1ms/step - loss: 0.5551 - accuracy: 0.7320
Epoch 432/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5543 - accuracy: 0.7326
Epoch 433/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5538 - accuracy: 0.7327
Epoch 434/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5534 - accuracy: 0.7324
Epoch 435/500
761/804 [===========================>..] - ETA: 0s - loss: 0.5528 - accuracy: 0.7331
Epoch 00435: saving model to checkpoints\weights-opt-1.435.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5542 - accuracy: 0.7324
Epoch 436/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5552 - accuracy: 0.7334
Epoch 437/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5547 - accuracy: 0.7334
Epoch 438/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5532 - accuracy: 0.7326
Epoch 439/500
804/804 [==================

804/804 [==============================] - 1s 1ms/step - loss: 0.5546 - accuracy: 0.7319
Epoch 494/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5548 - accuracy: 0.7317
Epoch 495/500
804/804 [==============================] - ETA: 0s - loss: 0.5551 - accuracy: 0.7315
Epoch 00495: saving model to checkpoints\weights-opt-1.495.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5551 - accuracy: 0.7315
Epoch 496/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5545 - accuracy: 0.7318
Epoch 497/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5539 - accuracy: 0.7324
Epoch 498/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5540 - accuracy: 0.7322
Epoch 499/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5549 - accuracy: 0.7317
Epoch 500/500
760/804 [===========================>..] - ETA: 0s - loss: 0.5557 - accuracy: 0.7306
Epoch 00500: saving model to checkpoints\weig

In [29]:
#load best weights and evaluate
maxValue = 0
epoch = 5
for i in range(10, 500, 5):
    acc = check_best_weights(application_df, drop = [ 'EIN', 'NAME', 'USE_CASE', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS', 'STATUS'],
         replace = [
            { 'col': 'APPLICATION_TYPE', 'threshold': 500 },
            { 'col': 'CLASSIFICATION', 'threshold': 1400 },
             { 'col': 'AFFILIATION', 'threshold': 100 }
         ],
          input_node = { 'units': 128, 'activation': 'relu' },
        hidden_nodes = [
            { 'units': 64, 'activation':  'relu'},
            { 'units': 32, 'activation':  'tanh'}
                       ],
        output_node = { 'units': 1, 'activation':  'sigmoid'},
        epochs = 500, save = False, path = f"checkpoints/weights-opt-1.{i}.hdf5",
        save_name = '1')
    
    if acc > maxValue:
        maxValue = acc
        epoch = i

268/268 - 0s - loss: 0.5615 - accuracy: 0.7254
Loss: 0.5615053176879883, Accuracy: 0.7253644466400146
268/268 - 0s - loss: 0.5587 - accuracy: 0.7264
Loss: 0.5587431192398071, Accuracy: 0.7264139652252197
268/268 - 0s - loss: 0.5573 - accuracy: 0.7275
Loss: 0.5573262572288513, Accuracy: 0.7274635434150696
268/268 - 0s - loss: 0.5587 - accuracy: 0.7273
Loss: 0.558662474155426, Accuracy: 0.7273469567298889
268/268 - 0s - loss: 0.5574 - accuracy: 0.7272
Loss: 0.557401716709137, Accuracy: 0.7272303104400635
268/268 - 0s - loss: 0.5567 - accuracy: 0.7277
Loss: 0.5567052960395813, Accuracy: 0.7276967763900757
268/268 - 0s - loss: 0.5575 - accuracy: 0.7271
Loss: 0.5575135946273804, Accuracy: 0.7271137237548828
268/268 - 0s - loss: 0.5580 - accuracy: 0.7283
Loss: 0.5580244660377502, Accuracy: 0.7282798886299133
268/268 - 0s - loss: 0.5567 - accuracy: 0.7277
Loss: 0.5566983819007874, Accuracy: 0.7276967763900757
268/268 - 0s - loss: 0.5596 - accuracy: 0.7249
Loss: 0.5595697164535522, Accuracy: 0

268/268 - 0s - loss: 0.5610 - accuracy: 0.7272
Loss: 0.5609782934188843, Accuracy: 0.7272303104400635
268/268 - 0s - loss: 0.5605 - accuracy: 0.7280
Loss: 0.5605341792106628, Accuracy: 0.7280466556549072
268/268 - 0s - loss: 0.5587 - accuracy: 0.7283
Loss: 0.558729350566864, Accuracy: 0.7282798886299133
268/268 - 0s - loss: 0.5650 - accuracy: 0.7268
Loss: 0.5650451183319092, Accuracy: 0.7267638444900513
268/268 - 0s - loss: 0.5635 - accuracy: 0.7278
Loss: 0.5635462403297424, Accuracy: 0.7278134226799011
268/268 - 0s - loss: 0.5592 - accuracy: 0.7289
Loss: 0.5591718554496765, Accuracy: 0.728863000869751
268/268 - 0s - loss: 0.5602 - accuracy: 0.7292
Loss: 0.5601826310157776, Accuracy: 0.7292128205299377
268/268 - 0s - loss: 0.5620 - accuracy: 0.7271
Loss: 0.5620132088661194, Accuracy: 0.7271137237548828
268/268 - 0s - loss: 0.5626 - accuracy: 0.7222
Loss: 0.562640368938446, Accuracy: 0.7222157716751099
268/268 - 0s - loss: 0.5632 - accuracy: 0.7292
Loss: 0.5631718039512634, Accuracy: 0.

In [30]:
# Save the best model
check_best_weights(application_df, drop = [ 'EIN', 'NAME', 'USE_CASE', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS', 'STATUS'],
         replace = [
            { 'col': 'APPLICATION_TYPE', 'threshold': 500 },
            { 'col': 'CLASSIFICATION', 'threshold': 1400 },
             { 'col': 'AFFILIATION', 'threshold': 100 }
         ],
          input_node = { 'units': 128, 'activation': 'relu' },
        hidden_nodes = [
            { 'units': 64, 'activation':  'relu'},
            { 'units': 32, 'activation':  'tanh'}
                       ],
        output_node = { 'units': 1, 'activation':  'sigmoid'},
        epochs = 500, save = True, path = f"checkpoints/weights-opt-1.{epoch}.hdf5",
        save_name = '1')

268/268 - 0s - loss: 0.5587 - accuracy: 0.7300
Loss: 0.5586708784103394, Accuracy: 0.7300291657447815


0.7300291657447815

### Best Attempt 2
Same as best attempt 1 except:
use fewer hidden neurons (based on reading that there should be less than 50% total hidden neurons than there is input neurons), include use_case, because model 1 may have been overfit since it performed better on the training data than the testing data.

In [31]:
run_model(application_df, drop = [ 'EIN', 'NAME', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS', 'STATUS'],
         replace = [
            { 'col': 'APPLICATION_TYPE', 'threshold': 500 },
            { 'col': 'CLASSIFICATION', 'threshold': 1400 },
             { 'col': 'AFFILIATION', 'threshold': 100 }
         ],
          input_node = { 'units': 128, 'activation': 'relu' },
        hidden_nodes = [
            { 'units': 30, 'activation':  'relu'},
            { 'units': 20, 'activation':  'tanh'}
                       ],
        output_node = { 'units': 1, 'activation':  'sigmoid'},
        epochs = 500, save = True, path = checkpoint_path_2,
        save_name = '2')

Epoch 1/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5722 - accuracy: 0.7184
Epoch 2/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5629 - accuracy: 0.7250
Epoch 3/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5616 - accuracy: 0.7261
Epoch 4/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5599 - accuracy: 0.7273
Epoch 5/500
794/804 [============================>.] - ETA: 0s - loss: 0.5590 - accuracy: 0.7268
Epoch 00005: saving model to checkpoints\weights-opt-2.05.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5590 - accuracy: 0.7268
Epoch 6/500
804/804 [==============================] - 1s 963us/step - loss: 0.5584 - accuracy: 0.7272
Epoch 7/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5577 - accuracy: 0.7277
Epoch 8/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5572 - accuracy: 0.7283
Epoch 9/500
804/804 [=====================

804/804 [==============================] - 1s 1ms/step - loss: 0.5483 - accuracy: 0.7331
Epoch 64/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5478 - accuracy: 0.7349
Epoch 65/500
760/804 [===========================>..] - ETA: 0s - loss: 0.5488 - accuracy: 0.7337
Epoch 00065: saving model to checkpoints\weights-opt-2.65.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5484 - accuracy: 0.7341
Epoch 66/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5482 - accuracy: 0.7348
Epoch 67/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5484 - accuracy: 0.7348
Epoch 68/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5482 - accuracy: 0.7343
Epoch 69/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5481 - accuracy: 0.7350
Epoch 70/500
785/804 [============================>.] - ETA: 0s - loss: 0.5476 - accuracy: 0.7340
Epoch 00070: saving model to checkpoints\weights-opt-

796/804 [============================>.] - ETA: 0s - loss: 0.5463 - accuracy: 0.7357
Epoch 00125: saving model to checkpoints\weights-opt-2.125.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5462 - accuracy: 0.7358
Epoch 126/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5463 - accuracy: 0.7354
Epoch 127/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5460 - accuracy: 0.7360
Epoch 128/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5459 - accuracy: 0.7355
Epoch 129/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5464 - accuracy: 0.7362
Epoch 130/500
802/804 [============================>.] - ETA: 0s - loss: 0.5459 - accuracy: 0.7357
Epoch 00130: saving model to checkpoints\weights-opt-2.130.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5459 - accuracy: 0.7357
Epoch 131/500
804/804 [==============================] - 1s 987us/step - loss: 0.5476 - accuracy: 

Epoch 186/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5486 - accuracy: 0.7351
Epoch 187/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5467 - accuracy: 0.7358
Epoch 188/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5470 - accuracy: 0.7355
Epoch 189/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5466 - accuracy: 0.7356
Epoch 190/500
764/804 [===========================>..] - ETA: 0s - loss: 0.5448 - accuracy: 0.7378
Epoch 00190: saving model to checkpoints\weights-opt-2.190.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5467 - accuracy: 0.7356
Epoch 191/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5465 - accuracy: 0.7361
Epoch 192/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5466 - accuracy: 0.7363
Epoch 193/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5464 - accuracy: 0.7360
Epoch 194/500
804/804 [====

804/804 [==============================] - 1s 1ms/step - loss: 0.5467 - accuracy: 0.7359
Epoch 249/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5464 - accuracy: 0.7354
Epoch 250/500
788/804 [============================>.] - ETA: 0s - loss: 0.5464 - accuracy: 0.7359
Epoch 00250: saving model to checkpoints\weights-opt-2.250.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5467 - accuracy: 0.7355
Epoch 251/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5496 - accuracy: 0.7286
Epoch 252/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5485 - accuracy: 0.7346
Epoch 253/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5471 - accuracy: 0.7361
Epoch 254/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5465 - accuracy: 0.7348
Epoch 255/500
778/804 [============================>.] - ETA: 0s - loss: 0.5463 - accuracy: 0.7356
Epoch 00255: saving model to checkpoints\weig

774/804 [===========================>..] - ETA: 0s - loss: 0.5460 - accuracy: 0.7372
Epoch 00310: saving model to checkpoints\weights-opt-2.310.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5474 - accuracy: 0.7359
Epoch 311/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5472 - accuracy: 0.7336
Epoch 312/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5468 - accuracy: 0.7346
Epoch 313/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5466 - accuracy: 0.7357
Epoch 314/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5469 - accuracy: 0.7353
Epoch 315/500
775/804 [===========================>..] - ETA: 0s - loss: 0.5473 - accuracy: 0.7369
Epoch 00315: saving model to checkpoints\weights-opt-2.315.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5466 - accuracy: 0.7371
Epoch 316/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5463 - accuracy: 0.

Epoch 371/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5481 - accuracy: 0.7356
Epoch 372/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5471 - accuracy: 0.7357
Epoch 373/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5471 - accuracy: 0.7360: 0s - loss: 0.5460 - accuracy: 0.
Epoch 374/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5466 - accuracy: 0.7362
Epoch 375/500
771/804 [===========================>..] - ETA: 0s - loss: 0.5466 - accuracy: 0.7363
Epoch 00375: saving model to checkpoints\weights-opt-2.375.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5466 - accuracy: 0.7363
Epoch 376/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5469 - accuracy: 0.7362
Epoch 377/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5484 - accuracy: 0.7355
Epoch 378/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5480 - accuracy: 

804/804 [==============================] - 1s 1ms/step - loss: 0.5463 - accuracy: 0.7360
Epoch 433/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5477 - accuracy: 0.7364
Epoch 434/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5464 - accuracy: 0.7358
Epoch 435/500
794/804 [============================>.] - ETA: 0s - loss: 0.5465 - accuracy: 0.7345
Epoch 00435: saving model to checkpoints\weights-opt-2.435.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5462 - accuracy: 0.7346
Epoch 436/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5485 - accuracy: 0.7327
Epoch 437/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5488 - accuracy: 0.7314
Epoch 438/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5481 - accuracy: 0.7343
Epoch 439/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5478 - accuracy: 0.7353
Epoch 440/500
754/804 [==================

804/804 [==============================] - 1s 1ms/step - loss: 0.5487 - accuracy: 0.7345
Epoch 495/500
751/804 [===========================>..] - ETA: 0s - loss: 0.5479 - accuracy: 0.7364
Epoch 00495: saving model to checkpoints\weights-opt-2.495.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5484 - accuracy: 0.7357
Epoch 496/500
804/804 [==============================] - 1s 962us/step - loss: 0.5488 - accuracy: 0.7350
Epoch 497/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5504 - accuracy: 0.7343
Epoch 498/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5487 - accuracy: 0.7362
Epoch 499/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5472 - accuracy: 0.7364
Epoch 500/500
766/804 [===========================>..] - ETA: 0s - loss: 0.5492 - accuracy: 0.7360
Epoch 00500: saving model to checkpoints\weights-opt-2.500.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5487 - accuracy: 

In [32]:
maxValue = 0
epoch = 5
for i in range(10, 500, 5):
    acc = check_best_weights(application_df, drop = [ 'EIN', 'NAME', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS', 'STATUS'],
         replace = [
            { 'col': 'APPLICATION_TYPE', 'threshold': 500 },
            { 'col': 'CLASSIFICATION', 'threshold': 1400 },
             { 'col': 'AFFILIATION', 'threshold': 100 }
         ],
          input_node = { 'units': 128, 'activation': 'relu' },
        hidden_nodes = [
            { 'units': 30, 'activation':  'relu'},
            { 'units': 20, 'activation':  'tanh'}
                       ],
        output_node = { 'units': 1, 'activation':  'sigmoid'},
        epochs = 500, save = False, path = f"checkpoints/weights-opt-2.{i}.hdf5",
        save_name = '2')
    
    if acc > maxValue:
        maxValue = acc
        epoch = i

268/268 - 0s - loss: 0.5593 - accuracy: 0.7238
Loss: 0.5593124032020569, Accuracy: 0.7238484025001526
268/268 - 0s - loss: 0.5575 - accuracy: 0.7257
Loss: 0.5575198531150818, Accuracy: 0.7257142663002014
268/268 - 0s - loss: 0.5558 - accuracy: 0.7269
Loss: 0.5557782649993896, Accuracy: 0.7268804907798767
268/268 - 0s - loss: 0.5559 - accuracy: 0.7257
Loss: 0.555860161781311, Accuracy: 0.7257142663002014
268/268 - 0s - loss: 0.5547 - accuracy: 0.7261
Loss: 0.5546884536743164, Accuracy: 0.726064145565033
268/268 - 0s - loss: 0.5550 - accuracy: 0.7279
Loss: 0.5549729466438293, Accuracy: 0.7279300093650818
268/268 - 0s - loss: 0.5557 - accuracy: 0.7257
Loss: 0.5556812286376953, Accuracy: 0.7257142663002014
268/268 - 0s - loss: 0.5535 - accuracy: 0.7273
Loss: 0.5535179376602173, Accuracy: 0.7273469567298889
268/268 - 0s - loss: 0.5551 - accuracy: 0.7280
Loss: 0.5550647377967834, Accuracy: 0.7280466556549072
268/268 - 0s - loss: 0.5539 - accuracy: 0.7282
Loss: 0.553938090801239, Accuracy: 0.

268/268 - 0s - loss: 0.5562 - accuracy: 0.7278
Loss: 0.5561923980712891, Accuracy: 0.7278134226799011
268/268 - 0s - loss: 0.5567 - accuracy: 0.7282
Loss: 0.5566503405570984, Accuracy: 0.7281632423400879
268/268 - 0s - loss: 0.5560 - accuracy: 0.7280
Loss: 0.5560468435287476, Accuracy: 0.7280466556549072
268/268 - 0s - loss: 0.5554 - accuracy: 0.7276
Loss: 0.5554375648498535, Accuracy: 0.727580189704895
268/268 - 0s - loss: 0.5581 - accuracy: 0.7276
Loss: 0.5580520033836365, Accuracy: 0.727580189704895
268/268 - 0s - loss: 0.5568 - accuracy: 0.7279
Loss: 0.5567831993103027, Accuracy: 0.7279300093650818
268/268 - 0s - loss: 0.5579 - accuracy: 0.7290
Loss: 0.5579476952552795, Accuracy: 0.7289795875549316
268/268 - 0s - loss: 0.5585 - accuracy: 0.7271
Loss: 0.5585442185401917, Accuracy: 0.7271137237548828
268/268 - 0s - loss: 0.5555 - accuracy: 0.7280
Loss: 0.5555099844932556, Accuracy: 0.7280466556549072
268/268 - 0s - loss: 0.5564 - accuracy: 0.7291
Loss: 0.5564170479774475, Accuracy: 0

In [33]:
check_best_weights(application_df, drop = [ 'EIN', 'NAME', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS', 'STATUS'],
         replace = [
            { 'col': 'APPLICATION_TYPE', 'threshold': 500 },
            { 'col': 'CLASSIFICATION', 'threshold': 1400 },
             { 'col': 'AFFILIATION', 'threshold': 100 }
         ],
          input_node = { 'units': 128, 'activation': 'relu' },
        hidden_nodes = [
            { 'units': 30, 'activation':  'relu'},
            { 'units': 20, 'activation':  'tanh'}
                       ],
        output_node = { 'units': 1, 'activation':  'sigmoid'},
        epochs = 500, save = True, path = f"checkpoints/weights-opt-2.{75}.hdf5",
        save_name = '2')

268/268 - 0s - loss: 0.5546 - accuracy: 0.7286
Loss: 0.5546016693115234, Accuracy: 0.7286297082901001


0.7286297082901001

### Best Attempt 3
Model 2 performed slightly worse than model 1, so model 3 is another varient on model 1 with the following changes: use teh optimized threshold for application type instead of classifiction.  remove the third layer, but use tahn in the remaining hidden layer.  Readd special considerations

In [34]:
run_model(application_df, drop = [ 'EIN', 'NAME', 'USE_CASE', 'INCOME_AMT', 'STATUS'],
         replace = [
            { 'col': 'APPLICATION_TYPE', 'threshold': 200 },
            { 'col': 'CLASSIFICATION', 'threshold': 1000 },
             { 'col': 'AFFILIATION', 'threshold': 100 }
         ],
          input_node = { 'units': 128, 'activation': 'relu' },
        hidden_nodes = [{ 'units': 64, 'activation':  'tanh'}],
        output_node = { 'units': 1, 'activation':  'sigmoid'},
        epochs = 500, save = True, path = checkpoint_path_3,
        save_name = '3')

Epoch 1/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5754 - accuracy: 0.7204
Epoch 2/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5665 - accuracy: 0.7248
Epoch 3/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5643 - accuracy: 0.7262
Epoch 4/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5638 - accuracy: 0.7273
Epoch 5/500
792/804 [============================>.] - ETA: 0s - loss: 0.5625 - accuracy: 0.7270
Epoch 00005: saving model to checkpoints\weights-opt-3.05.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5626 - accuracy: 0.7270
Epoch 6/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5622 - accuracy: 0.7269
Epoch 7/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5621 - accuracy: 0.7268
Epoch 8/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5615 - accuracy: 0.7283
Epoch 9/500
804/804 [=======================

804/804 [==============================] - 1s 1ms/step - loss: 0.5547 - accuracy: 0.7324
Epoch 64/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5548 - accuracy: 0.7325
Epoch 65/500
800/804 [============================>.] - ETA: 0s - loss: 0.5552 - accuracy: 0.7316
Epoch 00065: saving model to checkpoints\weights-opt-3.65.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5552 - accuracy: 0.7316
Epoch 66/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5546 - accuracy: 0.7326
Epoch 67/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5543 - accuracy: 0.7322
Epoch 68/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5550 - accuracy: 0.7322
Epoch 69/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5553 - accuracy: 0.7317
Epoch 70/500
771/804 [===========================>..] - ETA: 0s - loss: 0.5545 - accuracy: 0.7331
Epoch 00070: saving model to checkpoints\weights-opt-

792/804 [============================>.] - ETA: 0s - loss: 0.5538 - accuracy: 0.7322
Epoch 00125: saving model to checkpoints\weights-opt-3.125.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5540 - accuracy: 0.7322
Epoch 126/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5540 - accuracy: 0.7328
Epoch 127/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5543 - accuracy: 0.7316
Epoch 128/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5544 - accuracy: 0.7322
Epoch 129/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5539 - accuracy: 0.7329
Epoch 130/500
788/804 [============================>.] - ETA: 0s - loss: 0.5535 - accuracy: 0.7333
Epoch 00130: saving model to checkpoints\weights-opt-3.130.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5541 - accuracy: 0.7327
Epoch 131/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5537 - accuracy: 0.

Epoch 186/500
804/804 [==============================] - 1s 935us/step - loss: 0.5534 - accuracy: 0.7324
Epoch 187/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5539 - accuracy: 0.7322
Epoch 188/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5535 - accuracy: 0.7330
Epoch 189/500
804/804 [==============================] - 1s 979us/step - loss: 0.5536 - accuracy: 0.7329
Epoch 190/500
800/804 [============================>.] - ETA: 0s - loss: 0.5537 - accuracy: 0.7324
Epoch 00190: saving model to checkpoints\weights-opt-3.190.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5534 - accuracy: 0.7327
Epoch 191/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5536 - accuracy: 0.7328
Epoch 192/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5537 - accuracy: 0.7332
Epoch 193/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5535 - accuracy: 0.7330
Epoch 194/500
804/804 [

804/804 [==============================] - 1s 1ms/step - loss: 0.5530 - accuracy: 0.7322
Epoch 248/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5529 - accuracy: 0.7327
Epoch 249/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5528 - accuracy: 0.7330
Epoch 250/500
763/804 [===========================>..] - ETA: 0s - loss: 0.5537 - accuracy: 0.7325
Epoch 00250: saving model to checkpoints\weights-opt-3.250.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5530 - accuracy: 0.7332
Epoch 251/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5531 - accuracy: 0.7323
Epoch 252/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5531 - accuracy: 0.7320
Epoch 253/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5532 - accuracy: 0.7325
Epoch 254/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5532 - accuracy: 0.7327
Epoch 255/500
803/804 [==================

804/804 [==============================] - 1s 965us/step - loss: 0.5530 - accuracy: 0.7326
Epoch 309/500
804/804 [==============================] - 1s 951us/step - loss: 0.5528 - accuracy: 0.7323
Epoch 310/500
772/804 [===========================>..] - ETA: 0s - loss: 0.5537 - accuracy: 0.7324
Epoch 00310: saving model to checkpoints\weights-opt-3.310.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5531 - accuracy: 0.7332
Epoch 311/500
804/804 [==============================] - 1s 948us/step - loss: 0.5532 - accuracy: 0.7327
Epoch 312/500
804/804 [==============================] - 1s 948us/step - loss: 0.5530 - accuracy: 0.7329
Epoch 313/500
804/804 [==============================] - 1s 980us/step - loss: 0.5532 - accuracy: 0.7331
Epoch 314/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5529 - accuracy: 0.7330
Epoch 315/500
768/804 [===========================>..] - ETA: 0s - loss: 0.5530 - accuracy: 0.7333
Epoch 00315: saving model to checkp

804/804 [==============================] - 1s 1ms/step - loss: 0.5527 - accuracy: 0.7341
Epoch 370/500
779/804 [============================>.] - ETA: 0s - loss: 0.5539 - accuracy: 0.7325
Epoch 00370: saving model to checkpoints\weights-opt-3.370.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5531 - accuracy: 0.7331
Epoch 371/500
804/804 [==============================] - 1s 952us/step - loss: 0.5527 - accuracy: 0.7338
Epoch 372/500
804/804 [==============================] - 1s 950us/step - loss: 0.5529 - accuracy: 0.7334
Epoch 373/500
804/804 [==============================] - 1s 988us/step - loss: 0.5539 - accuracy: 0.7335
Epoch 374/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5526 - accuracy: 0.7334
Epoch 375/500
785/804 [============================>.] - ETA: 0s - loss: 0.5532 - accuracy: 0.7322
Epoch 00375: saving model to checkpoints\weights-opt-3.375.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5526 - accura

801/804 [============================>.] - ETA: 0s - loss: 0.5523 - accuracy: 0.7326
Epoch 00430: saving model to checkpoints\weights-opt-3.430.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5523 - accuracy: 0.7326
Epoch 431/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5520 - accuracy: 0.7325
Epoch 432/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5525 - accuracy: 0.7331
Epoch 433/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5528 - accuracy: 0.7324
Epoch 434/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5527 - accuracy: 0.7320
Epoch 435/500
793/804 [============================>.] - ETA: 0s - loss: 0.5523 - accuracy: 0.7334
Epoch 00435: saving model to checkpoints\weights-opt-3.435.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5525 - accuracy: 0.7332
Epoch 436/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5521 - accuracy: 0.

Epoch 491/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5523 - accuracy: 0.7334
Epoch 492/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5527 - accuracy: 0.7327
Epoch 493/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5528 - accuracy: 0.7339
Epoch 494/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5523 - accuracy: 0.7333
Epoch 495/500
759/804 [===========================>..] - ETA: 0s - loss: 0.5539 - accuracy: 0.7325
Epoch 00495: saving model to checkpoints\weights-opt-3.495.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5530 - accuracy: 0.7331
Epoch 496/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5531 - accuracy: 0.7331
Epoch 497/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5527 - accuracy: 0.7337
Epoch 498/500
804/804 [==============================] - 1s 1ms/step - loss: 0.5529 - accuracy: 0.7323
Epoch 499/500
804/804 [====

In [35]:
maxValue = 0
epoch = 5
for i in range(10, 500, 5):
    acc = check_best_weights(application_df, drop = [ 'EIN', 'NAME', 'USE_CASE', 'INCOME_AMT', 'STATUS'],
         replace = [
            { 'col': 'APPLICATION_TYPE', 'threshold': 200 },
            { 'col': 'CLASSIFICATION', 'threshold': 1000 },
             { 'col': 'AFFILIATION', 'threshold': 100 }
         ],
          input_node = { 'units': 128, 'activation': 'relu' },
        hidden_nodes = [{ 'units': 64, 'activation':  'tanh'}],
        output_node = { 'units': 1, 'activation':  'sigmoid'},
        epochs = 500, save = False, path = f"checkpoints/weights-opt-3.{i}.hdf5",
        save_name = '3')
    
    if acc > maxValue:
        maxValue = acc
        epoch = i

268/268 - 0s - loss: 0.5625 - accuracy: 0.7240
Loss: 0.5625137686729431, Accuracy: 0.7239649891853333
268/268 - 0s - loss: 0.5616 - accuracy: 0.7255
Loss: 0.5615808963775635, Accuracy: 0.7254810333251953
268/268 - 0s - loss: 0.5587 - accuracy: 0.7262
Loss: 0.5587329268455505, Accuracy: 0.7261807322502136
268/268 - 0s - loss: 0.5580 - accuracy: 0.7270
Loss: 0.5579942464828491, Accuracy: 0.7269970774650574
268/268 - 0s - loss: 0.5576 - accuracy: 0.7217
Loss: 0.5576079487800598, Accuracy: 0.7217492461204529
268/268 - 0s - loss: 0.5577 - accuracy: 0.7277
Loss: 0.5576621890068054, Accuracy: 0.7276967763900757
268/268 - 0s - loss: 0.5600 - accuracy: 0.7279
Loss: 0.5599623322486877, Accuracy: 0.7279300093650818
268/268 - 0s - loss: 0.5568 - accuracy: 0.7276
Loss: 0.5567815899848938, Accuracy: 0.727580189704895
268/268 - 0s - loss: 0.5576 - accuracy: 0.7280
Loss: 0.5576003193855286, Accuracy: 0.7280466556549072
268/268 - 0s - loss: 0.5575 - accuracy: 0.7263
Loss: 0.5574513077735901, Accuracy: 

268/268 - 0s - loss: 0.5567 - accuracy: 0.7279
Loss: 0.5566941499710083, Accuracy: 0.7279300093650818
268/268 - 0s - loss: 0.5583 - accuracy: 0.7249
Loss: 0.5583398938179016, Accuracy: 0.7248979806900024
268/268 - 0s - loss: 0.5565 - accuracy: 0.7275
Loss: 0.5564563870429993, Accuracy: 0.7274635434150696
268/268 - 0s - loss: 0.5578 - accuracy: 0.7230
Loss: 0.5577892661094666, Accuracy: 0.7230320572853088
268/268 - 0s - loss: 0.5566 - accuracy: 0.7234
Loss: 0.5565949082374573, Accuracy: 0.7233819365501404
268/268 - 0s - loss: 0.5576 - accuracy: 0.7252
Loss: 0.5575779676437378, Accuracy: 0.7252478003501892
268/268 - 0s - loss: 0.5562 - accuracy: 0.7273
Loss: 0.5562244057655334, Accuracy: 0.7273469567298889
268/268 - 0s - loss: 0.5578 - accuracy: 0.7285
Loss: 0.5577732920646667, Accuracy: 0.7285131216049194
268/268 - 0s - loss: 0.5566 - accuracy: 0.7290
Loss: 0.5566169619560242, Accuracy: 0.7289795875549316
268/268 - 0s - loss: 0.5572 - accuracy: 0.7279
Loss: 0.5572054386138916, Accuracy:

In [36]:
check_best_weights(application_df, drop = [ 'EIN', 'NAME', 'USE_CASE', 'INCOME_AMT', 'STATUS'],
         replace = [
            { 'col': 'APPLICATION_TYPE', 'threshold': 200 },
            { 'col': 'CLASSIFICATION', 'threshold': 1000 },
             { 'col': 'AFFILIATION', 'threshold': 100 }
         ],
          input_node = { 'units': 128, 'activation': 'relu' },
        hidden_nodes = [{ 'units': 64, 'activation':  'tanh'}],
        output_node = { 'units': 1, 'activation':  'sigmoid'},
        epochs = 500, save = True, path = f"checkpoints/weights-opt-3.{epoch}.hdf5",
        save_name = '3')

268/268 - 0s - loss: 0.5593 - accuracy: 0.7297
Loss: 0.5593422651290894, Accuracy: 0.72967928647995


0.72967928647995